Aqui são apresentados os métodos usados na limpeza, transformação, enriquecimento, as ferramentas que serão utilizadas na análise. Esse capítulo será desenvolvido em três partes: a) construção do dataset principal, com o enriquecimento, conexão com API meteorológica, construção do Jupyter notebook, importação das bibliotecas Pandas, Resquests, Matplot, Seaborn, Plotly Express;  b) tratamento e limpeza, convertendo os tipos, tratando dados ausentes, desenvolvendo engenharia de variáveis (feature engineering); c) análise exploratória dos dados.
Serão apresentadas os insights extraídos dessa análisedividida em três vertentes:
•	Temporal: existe uma sazonalidade? Quais dias de semana são mais críticos? Existe um horário com maior índice de acidentes?
•	Tipo de acidente: quais são mais comuns? Quais tipos de veículo mais se envolve em acidentes? Quais tipos tem mais vítimas?
•	Análise de severidade: qual a gravidade? Teve vítima fatal? Quais veículos causam mais vítimas? 


Importa as bibliotecas

In [1]:
# CARREGAR BIBLIOTECAS

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
Carrega dataset

In [2]:
# CARREGAR BANCO DE DADOS
# AJUSTAR COLUNAS

caminho_dados = 'dados/acidentes.csv'
separador_original = ';'
codificacao = 'utf-8'

caminho_dados_virgula = 'dados/acidentes-virgula.csv'
separador_virgula = ','

try:
    # 1. Carrega o CSV original usando o separador (;)
    df = pd.read_csv(caminho_dados, sep=separador_original, encoding=codificacao)

    print("CSV original lido com sucesso! Primeiras 5 linhas:")
    print(df.head())
    print("\nInformações do DataFrame original:")
    df.info()

    # 2. Salva o DataFrame em um novo arquivo CSV usando a vírgula (,) como separador
    # index=False evita que o Pandas grave o índice do DataFrame como uma nova coluna no CSV
    df.to_csv(caminho_dados_virgula, sep=separador_virgula, encoding=codificacao, index=False)

    print(f"\nArquivo ajustado salvo como '{caminho_dados_virgula}' com separador '{separador_virgula}'.")

except FileNotFoundError:
    print(f"Erro: Arquivo '{caminho_dados}' não encontrado. Verifique o caminho.")
except Exception as e:
    print(f"Ocorreu um erro: {e}")

CSV original lido com sucesso! Primeiras 5 linhas:
         data_extracao  predial1  queda_arr                 data  feridos  \
0  2025-06-01 01:33:13       0.0        0.0  2020-10-17 00:00:00        1   
1  2025-06-01 01:33:13     598.0        0.0  2020-01-01 00:00:00        1   
2  2025-06-01 01:33:13    1271.0        0.0  2020-01-01 00:00:00        1   
3  2025-06-01 01:33:13    1901.0        0.0  2020-01-02 00:00:00        2   
4  2025-06-01 01:33:13    3302.0        0.0  2020-01-02 00:00:00        1   

   feridos_gr  mortes  morte_post  fatais  auto  ...  longitude   latitude  \
0           0       0           0       0     3  ...   0.000000   0.000000   
1           0       0           0       0     0  ...        NaN        NaN   
2           1       0           0       0     1  ...        NaN        NaN   
3           0       0           0       0     0  ...        NaN        NaN   
4           0       0           0       0     1  ... -51.211535 -30.081535   

                 

In [25]:
# LIMPA NOMES COLUNAS

df = pd.read_csv('dados/acidentes-virgula.csv')

print("\n--- Nomes de todas as colunas no seu DataFrame: ---")
print(df.columns.tolist()) # Isso vai te dar a lista exata dos nomes das colunas

# Cria variável para armazenar coluna
coluna_data = 'data'

# Remove espaços em branco no início/fim dos nomes das colunas (boa prática)
df.columns = df.columns.str.strip()

# Re-imprime para ver se o nome da coluna de data foi limpo
print("\n--- Nomes das colunas após remover espaços em branco: ---")
print(df.columns.tolist())



--- Nomes de todas as colunas no seu DataFrame: ---
['data_extracao', 'predial1', 'queda_arr', 'data', 'feridos', 'feridos_gr', 'mortes', 'morte_post', 'fatais', 'auto', 'taxi', 'lotacao', 'onibus_urb', 'onibus_met', 'onibus_int', 'caminhao', 'moto', 'carroca', 'bicicleta', 'outro', 'cont_vit', 'ups', 'patinete', 'idacidente', 'longitude', 'latitude', 'log1', 'log2', 'tipo_acid', 'dia_sem', 'hora', 'noite_dia', 'regiao', 'consorcio']

--- Nomes das colunas após remover espaços em branco: ---
['data_extracao', 'predial1', 'queda_arr', 'data', 'feridos', 'feridos_gr', 'mortes', 'morte_post', 'fatais', 'auto', 'taxi', 'lotacao', 'onibus_urb', 'onibus_met', 'onibus_int', 'caminhao', 'moto', 'carroca', 'bicicleta', 'outro', 'cont_vit', 'ups', 'patinete', 'idacidente', 'longitude', 'latitude', 'log1', 'log2', 'tipo_acid', 'dia_sem', 'hora', 'noite_dia', 'regiao', 'consorcio']


In [30]:
# TRANSFORMA TIPO DATA
# EXCLUI DATAS INVÁLIDAS

coluna_data = 'data'
coluna_id = 'idacidente'

if coluna_data in df.columns and coluna_id in df.columns:

    print(f"\nColunas '{coluna_data}' e '{coluna_id}' encontradas no DataFrame.")

    # Converte a Coluna de Data para Datetime
    print(f"Tipo de dado atual da coluna '{coluna_data}': {df[coluna_data].dtype}")
    df[coluna_data] = pd.to_datetime(df[coluna_data], errors='coerce')
    print(f"Tipo de dado após a conversão: {df[coluna_data].dtype}")

    # Identifica e Imprimi ID do Acidente de Datas Inválidas
    # Cria um DataFrame contendo apenas as linhas onde a coluna de data é NaT (inválida)
    datas_invalidas_df = df[df[coluna_data].isna()]
    datas_invalidas_count = len(datas_invalidas_df)

    if datas_invalidas_count > 0:
        print(f"\nATENÇÃO: {datas_invalidas_count} datas inválidas (NaT) foram geradas na coluna '{coluna_data}'.")
        print(f"IDs de acidente para as datas inválidas:")
        print(f"DF datas invalidas\n{datas_invalidas_df}")
        # Itera sobre o DataFrame de datas inválidas e imprime o ID do acidente para cada linha
        for idx, row in datas_invalidas_df.iterrows():
            print(f"- ID do Acidente: {row[coluna_id]}")
            # Remove linha com data inválida
            primiera_linha = len(df)
            df = df.dropna(subset=[coluna_data])
            rows_removed = primiera_linha - len(df)
        print(f"\n{rows_removed} linhas com datas inválidas foram removidas do DataFrame.")
    else:
        print(f"\nColuna '{coluna_data}' convertida para datetime sem NaTs.")


    df['ano'] = df['data'].dt.year
    df['mes'] = df['data'].dt.month



Colunas 'data' e 'idacidente' encontradas no DataFrame.
Tipo de dado atual da coluna 'data': datetime64[ns]
Tipo de dado após a conversão: datetime64[ns]

Coluna 'data' convertida para datetime sem NaTs.


In [31]:
# Remove 'data_extração' e 'consorcio'

colunas_remover = ['data_extracao', 'consorcio']
colunas_existentes = [col for col in colunas_remover if col in df.columns]

if colunas_existentes:
    df = df.drop(columns=colunas_existentes)
    print(f"Colunas extracao e consorcio removidas.")

In [32]:
# Filtra o DataFrame por Ano
ano_inicio = 2020
ano_fim = 2024

# A filtragem agora usa o coluna_data
df_filtrado = df[
    (df[coluna_data].dt.year >= ano_inicio) &
    (df[coluna_data].dt.year <= ano_fim)
]

print(f"\n--- DataFrame Filtrado por Ano ({ano_inicio} a {ano_fim}) ---")
print(f"Número de linhas no DataFrame original: {len(df)}")
print(f"Número de linhas no DataFrame filtrado: {len(df_filtrado)}")

print("\n>>> Primeiras 5 linhas do DataFrame Filtrado:")
print(df_filtrado.head())
print("\n>>> Contagem de registros por ano (filtrado):")
print(df_filtrado[coluna_data].dt.year.value_counts().sort_index())



--- DataFrame Filtrado por Ano (2020 a 2024) ---
Número de linhas no DataFrame original: 69520
Número de linhas no DataFrame filtrado: 66120

>>> Primeiras 5 linhas do DataFrame Filtrado:
   predial1  queda_arr       data  feridos  feridos_gr  mortes  morte_post  \
0       0.0        0.0 2020-10-17        1           0       0           0   
1     598.0        0.0 2020-01-01        1           0       0           0   
2    1271.0        0.0 2020-01-01        1           1       0           0   
3    1901.0        0.0 2020-01-02        2           0       0           0   
4    3302.0        0.0 2020-01-02        1           0       0           0   

   fatais  auto  taxi  ...                log1              log2  \
0       0     3     0  ...    R MARCOS MOREIRA  R GASTON ENGLERT   
1       0     0     1  ...  AV BENTO GONCALVES               NaN   
2       0     1     0  ...    AV INDEPENDENCIA               NaN   
3       0     0     0  ...    AV EDUARDO PRADO               NaN   
4 

In [23]:
print("\n--- Informações Gerais do Dataset Após Limpeza e Conversão ---")
df_filtrado.info()
print("\n--- Contagem de Valores Ausentes Após Limpeza e Conversão ---")
print(df_filtrado.isnull().sum())


--- Informações Gerais do Dataset Após Limpeza e Conversão ---
<class 'pandas.core.frame.DataFrame'>
Index: 66120 entries, 0 to 69344
Data columns (total 35 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   predial1    62312 non-null  float64       
 1   queda_arr   66120 non-null  float64       
 2   data        66120 non-null  datetime64[ns]
 3   feridos     66120 non-null  int64         
 4   feridos_gr  66120 non-null  int64         
 5   mortes      66120 non-null  int64         
 6   morte_post  66120 non-null  int64         
 7   fatais      66120 non-null  int64         
 8   auto        66120 non-null  int64         
 9   taxi        66120 non-null  int64         
 10  lotacao     66120 non-null  int64         
 11  onibus_urb  66120 non-null  int64         
 12  onibus_met  66120 non-null  int64         
 13  onibus_int  66120 non-null  int64         
 14  caminhao    66120 non-null  int64         
 15  moto       

In [ ]:
# TRATA 'LONGITUDE' E 'LATITUDE'
